In [4]:
import os
import pandas as pd
import numpy as np
import cv2
from kaggle.api.kaggle_api_extended import KaggleApi
from zipfile import ZipFile
from tqdm import tqdm # 진행 상황을 시각적으로 보여주기 위한 라이브러리
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

print("✅ 라이브러리 임포트 완료!")

✅ 라이브러리 임포트 완료!


In [5]:
# Kaggle API 설정
kaggle_api = KaggleApi()
kaggle_api.authenticate()

# 데이터셋 다운로드 경로 및 파일 이름 설정
dataset_name = 'gauravduttakiit/pharmaceutical-drug-recognition'
download_path = '.'
zip_file_name = 'pharmaceutical-drug-recognition.zip'

# 데이터셋이 이미 준비되지 않았다면 다운로드 및 압축 해제
if not os.path.exists('train'):
    print("데이터셋 다운로드를 시작합니다...")
    kaggle_api.dataset_download_files(dataset_name, path=download_path, quiet=False)
    
    print(f"'{zip_file_name}' 파일 압축 해제를 시작합니다...")
    with ZipFile(zip_file_name, 'r') as zip_ref:
        zip_ref.extractall(download_path)
    
    os.remove(zip_file_name)
    print("✅ 데이터셋 준비 완료!")
else:
    print("✅ 데이터셋이 이미 준비되었습니다.")

✅ 데이터셋이 이미 준비되었습니다.


### 여러 개의 템플릿 사용하기
- 가장 간단하면서도 효과적인 방법입니다. 각 약품마다 하나의 이미지만을 템플릿으로 사용하는 대신, 여러 개의 다른 이미지를 템플릿으로 등록하는 것입니다.
- 방법: Training_set.csv에서 각 약품마다 1개가 아닌, 5~10개의 이미지를 템플릿으로 저장합니다.
- 예측: 테스트 이미지를 예측할 때, 해당 약품의 모든 템플릿과 비교하여 그중 가장 높게 나온 유사도 점수를 그 약품의 최종 점수로 사용합니다.
- 효과: 같은 약이라도 약간씩 다른 각도, 조명, 인쇄 상태를 커버할 수 있어 정확도가 향상됩니다.
<br>

### 크기 변화에 대응하기
- 테스트 이미지와 템플릿의 크기가 다르면 매칭이 잘되지 않습니다. 이를 해결하기 위해 테스트 이미지의 크기를 여러 버전으로 만든 후 매칭을 시도할 수 있습니다. 이를 이미지 피라미드(Image Pyramid) 기법이라고 합니다.
- 방법: 테스트 이미지를 90%, 100%, 110% 등 여러 비율로 리사이즈하여 여러 장의 이미지를 만듭니다.
- 예측: 각 크기별 테스트 이미지에 대해 모두 템플릿 매칭을 수행하고, 모든 결과를 통틀어 가장 높은 유사도 점수를 최종 결과로 선택합니다.
- 효과: 약품이 사진에서 조금 더 크거나 작게 찍혔을 때도 잘 찾아낼 수 있습니다.
<br>

### 회전 변화에 대응하기
- 템플릿 매칭은 이미지 회전에 매우 취약합니다. 이를 보완하려면 템플릿 자체를 여러 각도로 회전시켜 준비해두는 것입니다.
- 방법: 각 템플릿 이미지를 5도, 10도, 15도 등 여러 각도로 미리 회전시킨 이미지를 모두 저장해 둡니다.
- 예측: 테스트 이미지를 이 모든 회전된 템플릿들과 비교하여 가장 높은 점수를 찾습니다.
- 효과: 약품이 약간 비뚤게 놓여있는 경우에도 대응할 수 있게 됩니다.
- 참고: 2번(크기)과 3번(회전) 방법을 모두 적용하면 경우의 수가 너무 많아져 예측 시간이 매우 길어질 수 있습니다.
<br>

### 전처리 강화: 윤곽선(Edge)사용하기
- 이미지의 색상이나 밝기 변화에 덜 민감하게 만들기 위해, 원본 이미지 대신 윤곽선(Edge)만 추출한 이미지로 템플릿 매칭을 할 수 있습니다.
- 방법: 캐니 엣지 디텍션(Canny Edge Detection) 알고리즘을 사용하여 템플릿과 테스트 이미지 모두를 윤곽선 이미지로 변환합니다.
- 예측: 이 윤곽선 이미지끼리 템플릿 매칭을 수행합니다.
- 효과: 조명이나 그림자의 영향을 거의 받지 않고, 오직 약품의 모양(shape)에만 집중하여 비교하므로 성능이 향상될 수 있습니다.

## 평균 + 회전
각 약품의 평균 윤곽선 템플릿을 만들고, 이를 여러 각도로 회전시켜 최종 템플릿 세트를 준비합니다.

In [6]:
# --- 템플릿 생성을 위한 상수 정의 ---
DATA_DIR = 'train'
CSV_FILE = 'Training_set.csv'
IMG_SIZE = (128, 128)
NUM_IMAGES_FOR_AVERAGE = 10  # 평균 템플릿을 만들기 위해 사용할 이미지 개수
ROTATION_ANGLES = [-30, -20, -10, 0, 10, 20, 30] # 템플릿을 회전시킬 각도
AVERAGE_THRESHOLD_PERCENT = 0.3 # 평균 이미지에서 윤곽선으로 판단할 임계값 (30%)

# --- 데이터 분할 ---
df_full = pd.read_csv(CSV_FILE)
df_template_creation, df_evaluation = train_test_split(
    df_full, test_size=0.2, random_state=42, stratify=df_full['label'])

# --- 평균 윤곽선 + 회전 템플릿 생성 ---
drug_classes = df_full['label'].unique()
# 구조: {'약품이름': [(각도1, 회전템플릿1), (각도2, 회전템플릿2), ...]}
templates = {drug: [] for drug in drug_classes}

print("고급 템플릿(평균 윤곽선 + 회전) 생성을 시작합니다...")

for drug in tqdm(drug_classes, desc="템플릿 생성 중"):
    # 1. 평균 윤곽선 템플릿 생성
    template_rows = df_template_creation[df_template_creation['label'] == drug].head(NUM_IMAGES_FOR_AVERAGE)
    canny_images = []
    for index, row in template_rows.iterrows():
        path = os.path.join(DATA_DIR, row['filename'])
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, IMG_SIZE)
        canny_images.append(cv2.Canny(img, 50, 150))

    # 여러 윤곽선 이미지를 평균내어 '확률 맵' 생성
    mean_canny_image = np.mean(canny_images, axis=0)
    # 특정 임계값 이상인 픽셀만 최종 윤곽선으로 선택
    threshold_value = 255 * AVERAGE_THRESHOLD_PERCENT
    _, average_edge_template = cv2.threshold(mean_canny_image, threshold_value, 255, cv2.THRESH_BINARY)
    average_edge_template = average_edge_template.astype(np.uint8)

    # 2. 평균 템플릿을 여러 각도로 회전하여 저장
    center = (IMG_SIZE[0] // 2, IMG_SIZE[1] // 2)
    for angle in ROTATION_ANGLES:
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        rotated_template = cv2.warpAffine(average_edge_template, M, IMG_SIZE, flags=cv2.INTER_LINEAR)
        templates[drug].append(rotated_template)

print("✅ 모든 고급 템플릿 생성을 완료했습니다.")

고급 템플릿(평균 윤곽선 + 회전) 생성을 시작합니다...


템플릿 생성 중: 100%|██████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 80.24it/s]

✅ 모든 고급 템플릿 생성을 완료했습니다.


In [7]:
# --- 평가를 위한 상수 정의 ---
SCALES = [0.9, 1.0, 1.1] # 이미지 피라미드 스케일

correct_predictions = 0
total_predictions = len(df_evaluation)

print("\n이미지 피라미드와 회전 템플릿을 사용하여 종합 평가를 시작합니다...")

for index, row in tqdm(df_evaluation.iterrows(), total=total_predictions, desc="평가 진행 중"):
    test_image_path = os.path.join(DATA_DIR, row['filename'])
    true_label = row['label']
    
    test_image_gray = cv2.imread(test_image_path, cv2.IMREAD_GRAYSCALE)
    if test_image_gray is None: continue

    overall_best_score = -1
    predicted_label = None

    # 1. 이미지 피라미드: 테스트 이미지를 여러 크기로 순회
    for scale in SCALES:
        width = int(IMG_SIZE[0] * scale)
        height = int(IMG_SIZE[1] * scale)
        resized_test_image = cv2.resize(test_image_gray, (width, height))
        test_image_canny = cv2.Canny(resized_test_image, 50, 150)

        # 2. 모든 약품 클래스 순회
        for drug_name, rotated_templates in templates.items():
            # 3. 모든 회전 템플릿 순회
            for rotated_template in rotated_templates:
                # 템플릿이 테스트 이미지보다 크면 매칭 불가
                if rotated_template.shape[0] > test_image_canny.shape[0] or rotated_template.shape[1] > test_image_canny.shape[1]:
                    continue

                result = cv2.matchTemplate(test_image_canny, rotated_template, cv2.TM_CCOEFF_NORMED)
                _, max_val, _, _ = cv2.minMaxLoc(result)
                
                # 전체 스케일과 회전을 통틀어 최고 점수 갱신
                if max_val > overall_best_score:
                    overall_best_score = max_val
                    predicted_label = drug_name
            
    if predicted_label == true_label:
        correct_predictions += 1

accuracy = (correct_predictions / total_predictions) * 100

print("\n✅ 평가 완료!")
print(f"정확하게 예측한 이미지 수: {correct_predictions} / {total_predictions}")
print(f"최종 정확도 (Accuracy): {accuracy:.2f}%")


이미지 피라미드와 회전 템플릿을 사용하여 종합 평가를 시작합니다...


평가 진행 중: 100%|████████████████████████████████████████████████████████████████| 1400/1400 [00:19<00:00, 71.47it/s]


✅ 평가 완료!
정확하게 예측한 이미지 수: 160 / 1400
최종 정확도 (Accuracy): 11.43%
